<a href="https://colab.research.google.com/github/wenom1384/ME536_Project/blob/main/ME536_Project_Neat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MY INTERACTIVE ROBOT

First three code section are;

1. Loading package
2. Getting important files, functions 
3. Getting all imports for the code

In [ ]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 42kB/s 


In [ ]:
!rm *.jpg
!rm *.py
!rm *.npy
!rm *.xml
!rm *.txt
!wget https://github.com/wenom1384/ME536_Project/raw/main/MYCODEUTIL.py
!wget https://github.com/wenom1384/ME536_Project/raw/main/M_encodings.npy
!wget https://github.com/wenom1384/ME536_Project/raw/main/namesgrp.npy
!wget https://github.com/wenom1384/ME536_Project/raw/main/ave_list.npy
!wget https://github.com/wenom1384/ME536_Project/raw/main/PC.npy
!wget https://github.com/wenom1384/ME536_Project/raw/main/ave_list_names.txt
!wget https://github.com/wenom1384/ME536_Project/raw/main/Myfunctions.py

In [ ]:
import cv2
import sys
from google.colab.patches import cv2_imshow
import face_recognition
import numpy as np
from scipy.linalg import orth
import dlib
import argparse
import pickle
import os
from IPython.display import clear_output
import matplotlib.pyplot as plt
from numpy.linalg import matrix_rank  as rank
from time import sleep
from MYCODEUTIL import SVD536
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from scipy.spatial import distance
from IPython.display import Image

import PIL
import io
from base64 import b64decode, b64encode

from MYCODEUTIL import js_to_image
from MYCODEUTIL import bbox_to_bytes
from MYCODEUTIL import video_stream
from MYCODEUTIL import video_frame
import MYCODEUTIL as util
import Myfunctions as myfunc
from Myfunctions  import hellofriend
from Myfunctions  import myfacedetector

# RESET/BOOT MEMORY
Below section reset or boot the memory of the robot

In [ ]:


#ave_list created.
ave_list=np.load('ave_list.npy').tolist()

#PC created.
PC=np.load('PC.npy')

#ave_list_names created.
with open('ave_list_names.txt') as f:
  ave_list_names = f.readlines()
ave_list_names = [x.strip() for x in ave_list_names] 

#M >> matrix of encodings are created.
M=np.load('M_encodings.npy')

#namesgrp >> a mapping for encodings in the M is created.
namesgrp=np.load('namesgrp.npy')

# TO START INTERACTION RUN BELOW CODE

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
eager = 0
permission='0'
new_encodings=[]
count2know=0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # get face region coordinates
    #faces = face_cascade.detectMultiScale(gray)
    boxes,name,score,res_list,unknown=myfacedetector(img,ave_list,ave_list_names,PC)

    # get face bounding box for overlay
    #for (x,y,w,h) in faces:
      #bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    count=0
    for t,r,b,l in boxes:
      bbox_array=cv2.rectangle(bbox_array, (l, t), (r,b), (0, 255, 0), 2)
    
      bbox_array=cv2.putText(bbox_array, name[count], (l,t), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
      bbox_array=cv2.putText(bbox_array,'res'+np.array2string(res_list[count],precision=2)+'scr'+np.array2string(score[count],precision=2), (l,t-20), cv2.FONT_HERSHEY_SIMPLEX,0.55, (0, 255, 0), 2)
      if unknown==1:
        bbox_array=cv2.putText(bbox_array,'kim bu', (200,340), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
      count+=1

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    if permission and not permission=='0':
      if count2know==0: print('Move your head around pls...')
      rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      #boxes = face_recognition.face_locations(rgb,model='hog')
      #print(boxes)
      new_encodings.append (face_recognition.face_encodings(rgb, boxes))
      #print(len(new_encodings))
      count2know+=1

      if count2know==25:    
        ave_list,ave_list_names,M,namesgrp,PC=hellofriend(permission,new_encodings,ave_list,ave_list_names,M,namesgrp)
        permission='0'
        new_encodings=[]
        count2know=0

      

    if unknown==1 and permission=='0':
      eager=eager+1
      #print(eager)
 
      
       
    if eager>10 and  permission=='0':

      permission= input('If you want me to know you type your name and show yourself else type 0:\n') 

      eager=0
      print(permission)  

